In [ ]:
!pip install dlib

In [24]:
import os
import dlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Path to the folders containing real and fake images
real_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
fake_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [19]:
# Load and preprocess images
def load_and_preprocess_images(folder, max_images=3000):
    images = []
    count = 0
    for filename in os.listdir(folder):
        if count >= max_images:
            break
        print(count)
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = dlib.load_rgb_image(img_path)
            images.append(img)
            count += 1
    return images

In [20]:
# Load the face pose estimator from DLIB
face_pose_estimator = dlib.shape_predictor("/content/gdrive/MyDrive/shape_predictor_68_face_landmarks.dat")

In [ ]:
# Load and preprocess real and fake images
real_images = load_and_preprocess_images(real_folder_path)
fake_images = load_and_preprocess_images(fake_folder_path)

In [ ]:
# print(Flatten()(real_images[0]))
# print(real_images[0].flatten())

In [8]:
image_height, image_width, num_channels = real_images[0].shape

In [9]:
# Combine real and fake images and create labels
all_images = np.concatenate((real_images, fake_images), axis=0)
labels = np.concatenate((np.ones(len(real_images)), np.zeros(len(fake_images))), axis=0)

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [10]:
# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(all_images, labels, test_size=0.2, random_state=42)

def extract_face_pose_features(images):
    features = []
    for img in images:
        face_landmarks = face_pose_estimator(img, dlib.rectangle(0, 0, img.shape[1], img.shape[0]))
        pose_feature = np.array([(point.x, point.y) for point in face_landmarks.parts()])
        features.append(pose_feature)
    return features

In [11]:
train_features = extract_face_pose_features(train_images)
val_features = extract_face_pose_features(val_images)

In [12]:
# Flatten the feature arrays
train_features_flatten = np.array([feature.flatten() for feature in train_features])
val_features_flatten = np.array([feature.flatten() for feature in val_features])

In [13]:
# Define your neural network architecture
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [14]:
# Define model architecture
feature_dim = train_features_flatten.shape[1]
model = build_model((feature_dim,))

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 32
epochs = 10

model.fit(train_features_flatten, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_features_flatten, val_labels))

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(val_features_flatten, val_labels)
print("Test accuracy:", test_accuracy)